In [2]:
# Imports

# Matemaricos, mapiulacao de dados e funcoes
import numpy as np
import pandas as pd
import time
# Desligando alertas de chained_assignment
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

# Chamadas e tratamentos de dados e arquivos
import re
import requests
from pathlib import Path

# Desativacao de warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# GetDate
from datetime import date

In [3]:
# Carregamento da listagem de propostas
listagem = pd.DataFrame(pd.read_excel('../Listagem/listagem_13_1.xlsx'))
# Limpeza de colunas vazias
listagem = listagem.drop(listagem.columns[[1,3,5,7,9]], axis=1)
# Limpeza de linhas varias (NaN)
listagem = listagem.dropna()

listagem['Projeto'] = listagem['Projeto'].astype(int)
listagem['Ano'] = listagem['Ano'].astype(int)
listagem = listagem.rename(columns = {'Situação':'Situacao'})

In [5]:
'''Criacao das colunas: 
codigo do tipo
id do projeto
data de proposicao do projeto
status da tramitacao
conferência da situacao
'''
listagem['Codigo_Tipo'] = ''
listagem['Id_Projeto'] = ''
listagem['Data_Projeto'] = ''
listagem['Tramitacao_Encerrada'] = ''
listagem['Aprovado'] = '' 

#listagem['Tramitacao'] = '' # UNIAO DOS DOIS CAMPOS ANTERIORES EM REVISAO

In [6]:
# Preenchimento do campo Codigo_Tipo
unknown_count = 0

for i in listagem.index:
    #print(listagem["Tipo"][i])
    if(listagem["Tipo"][i] == "Requerimento"):
        listagem['Codigo_Tipo'][i] = 'REQ'
    elif(listagem["Tipo"][i] == "Projeto de Lei"):
        listagem['Codigo_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Lei Complementar"):
        listagem['Codigo_Tipo'][i] = 'PLEIC'
    elif(listagem["Tipo"][i] == "Representação"):
        listagem['Codigo_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Resolução"):
        listagem['Codigo_Tipo'][i] = 'PRES'
    elif(listagem["Tipo"][i] == "Pedido de Informação"):
        listagem['Codigo_Tipo'][i] = 'PEDIF'
    elif(listagem["Tipo"][i] == "Moção"):
        listagem['Codigo_Tipo'][i] = 'MOC'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei)"):
        listagem['Codigo_Tipo'][i] = 'MSGPL'
    elif(listagem["Tipo"][i] == "Indicação"):
        listagem['Codigo_Tipo'][i] = 'IND'
    elif(listagem["Tipo"][i] == "Projeto de Decreto Legislativo"):
        listagem['Codigo_Tipo'][i] = 'PDL'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo"):
        listagem['Codigo_Tipo'][i] = 'MSGEX'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei Complementar)"):
        listagem['Codigo_Tipo'][i] = 'MSGPC'
    elif(listagem["Tipo"][i] == "Prestação de Contas"):
        listagem['Codigo_Tipo'][i] = 'PTCTS'
    elif(listagem["Tipo"][i] == "Projeto de Emenda à Lei Orgânica"):
        listagem['Codigo_Tipo'][i] = 'PELOR'
    elif(listagem["Tipo"][i] == "Projeto de Lei de Iniciativa Popular"):
        listagem['Codigo_Tipo'][i] = 'PLEIP'
    elif(listagem["Tipo"][i] == "Projeto de Resolução - Comissão de Exame Denúncia"):
        listagem['Codigo_Tipo'][i] = 'PRCED'
    elif(listagem["Tipo"][i] == "Proposta de Revisão da Lei Orgânica Municipal"):
        listagem['Codigo_Tipo'][i] = 'PRLOM'
    elif(listagem["Tipo"][i] == "Proposta de Revisão do Regimento Interno"):
        listagem['Codigo_Tipo'][i] = 'PRRI'
    elif(listagem["Tipo"][i] == "Requerimento Popular"):
        listagem['Codigo_Tipo'][i] = 'RPO'
    elif(listagem["Tipo"][i] == "Requerimento de CPI"):
        listagem['Codigo_Tipo'][i] = 'RCPI'
    elif(listagem["Tipo"][i] == "Pedido de Informação FiscalizaJF"):
        listagem['Codigo_Tipo'][i] = 'PEDFF'
    elif(listagem["Tipo"][i] == "Emenda"):
        listagem['Codigo_Tipo'][i] = 'EMENDA'
    else:
        listagem['Codigo_Tipo'][i] = 'DESCONHECIDO'
        unknown_count += 1

print(f'Fim do processo. Traducao realizada com {unknown_count} registros nao encontrados')

Fim do processo. Traducao realizada com 0 registros nao encontrados


In [7]:
# Preenchimento do campo Id_Projeto
count = 1
failed_req= 0
failed = False
id_projeto = ''
max_failed_req = 5
notfound_count = 0

while count > 0:
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'): 
            id_projeto = ''
            failed = False
            # Variaveis de requisicao
            num_proj = int(listagem['Projeto'][i])
            ano_proj = int(listagem['Ano'][i])
            tipo_proj = listagem['Codigo_Tipo'][i]

            try:
                # Requisicao para o portal de propostas da Camara
                query = {"numero": num_proj, "ano": ano_proj, "codtipo": tipo_proj}
                #query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    id_projeto = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                except:
                    try:
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                        id_projeto = re.search('PROPOSICAON.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
                    except:
                        id_projeto = 'Nao Encontrado'
                        notfound_count += 1
            except:
                #Assinala que houve falha
                failed = True
                time.sleep(180)
                    
            listagem['Id_Projeto'][i] = id_projeto
            resposta.close()
            time.sleep(6)
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break

    if failed_req > max_failed_req:
        break   
        
    count = 0
    
    for i in listagem.index:
        if listagem['Id_Projeto'][i] == '' and listagem['Codigo_Tipo'][i] not in ('DESCONHECIDO','EMENDA'):
            count +=1

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}')
else:
    print('Fim do processo')

Fim do processo


In [28]:
i=4

#listagem['Id_Projeto'][4] = 117403
#listagem['Data_Projeto'][i] = ''
#listagem['Tramitacao_Encerrada'][i] = ''
#listagem['Aprovado'][i] = ''           

display(listagem)

,Projeto,Ano,Tipo,Autor,Ementa,Situacao,Codigo_Tipo,Id_Projeto,Data_Projeto,Tramitacao_Encerrada,Aprovado
0,5842,2013,Requerimento,"Dr. Fiorilo, Luiz Otávio Fernandes Coelho - Pa...",Solicita junto a Secretaria de Educação inclus...,Arquivada,REQ,87441,28/08/2020,SIM,SIM
2,5841,2013,Requerimento,"Dr. Fiorilo, Luiz Otávio Fernandes Coelho - Pa...",Solicita junto a Secretaria de Esportes e Laze...,Arquivada,REQ,118584,29/09/2022,NAO,NAO
4,5840,2013,Requerimento,"Dr. Fiorilo, Luiz Otávio Fernandes Coelho - Pa...","Solicita junto a Secretaria de Educação, Admin...",Arquivada,REQ,117403,25/08/2022,SIM,SIM


In [27]:
# Preenchimento do campo Data_Projeto
count = 1
failed_req= 0
failed = False
max_failed_req = 5
notfound_count = 0

data_projeto = ''
notfound_count_dt = 0

tramitacao_encerrada = ''
notfound_count_tram = 0

aprovado = ''
notfound_count_aprv = 0

while count > 0:
    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '' or listagem['Tramitacao_Encerrada'][i] == '' or listagem['Aprovado'][i] == '': 
            data_projeto = ''
            tramitacao_encerrada = ''
            aprovado = ''

            failed = False
            # Variaveis de requisicao
            id_proj = listagem['Id_Projeto'][i]
            
            try:
                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
                
                #Identificacao da data de registro do projeto
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                except:
                    data_projeto = 'Data nao encontrada'
                    notfound_count_dt += 1
                
                
                #Identificacao do encerramento da tramitacao do projeto
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    #tramitacao_encerrada = re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:(.+?)\t\t\t\t\t\t', retorno).group(1)
                    if(re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:SIM', retorno)):
                        tramitacao_encerrada = 'SIM'
                    else:
                        tramitacao_encerrada = 'NAO'
                except:
                    tramitacao_encerrada = 'Informacao nao encontrada'
                    notfound_count_tram+=1
                    
                    
                #Identificacao da aprovacao do projeto
                if 'Aprovad' in listagem['Situacao'][i] or 'Trasformado em Norma Jurídica' in listagem['Situacao'][i]:
                    aprovado = 'SIM'
                elif 'Retirad' in listagem['Situacao'][i] or 'Rejeitad' in listagem['Situacao'][i] or 'Vetado' in listagem['Situacao'][i]:# era 'Vetado Totalmente'\
                    aprovado = 'NAO'
                elif tramitacao_encerrada == 'NAO': 
                    aprovado = 'TRAMITACAO'
                #elif listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF'):
                #    if listagem['Situacao'][i] == 'Arquivada': 
                #        aprovado = 'NAO'        
                #    else:        
                #        aprovado = 'SIM'
                #elif listagem['Codigo_Tipo'][i] in ('REQ', 'MOC', 'REP', 'PEDIF'): 
                else:
                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                        if(re.search('APROVAD', retorno)):
                            aprovado = 'SIM'
                        else:
                            aprovado = 'NAO'
                    except:
                        aprovado = 'Informacao nao encontrada'
                        notfound_count_aprv += 1
                
            except:
                failed = True
                time.sleep(180)

            listagem['Data_Projeto'][i] = data_projeto
            listagem['Tramitacao_Encerrada'][i] = tramitacao_encerrada
            listagem['Aprovado'][i] = aprovado
            
            resposta.close()
            time.sleep(6)
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '' and listagem['Id_Projeto'][i] != '':
            count +=1

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}')

Fim do processo. Nao encontrados = 0


## Antigos

In [ ]:
# Preenchimento do campo Data_Projeto
count = 1
failed_req= 0
failed = False
data_projeto = ''
max_failed_req = 5
notfound_count = 0

while count > 0:
    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '': 
            data_projeto = ''
            failed = False
            # Variaveis de requisicao
            id_proj = listagem['Id_Projeto'][i]
            
            try:
                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)
                                
                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                except:
                    data_projeto = 'Data nao encontrada'
                    notfound_count += 1
            except:
                failed = True

            listagem['Data_Projeto'][i] = data_projeto
            resposta.close()
            time.sleep(5)
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Data_Projeto'][i] == '':
            count +=1

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}')

In [ ]:
# Preenchimento do campo Tramitacao_Encerrada
count = 1
failed_req= 0
failed = False
tramitacao_encerrada = ''
max_failed_req = 5
notfound_count = 0

while count > 0:
    for i in listagem.index:
        if listagem['Tramitacao_Encerrada'][i] == '': 
            tramitacao_encerrada = ''
            failed = False
            
            try:
                # Variaveis de requisicao
                id_proj = listagem['Id_Projeto'][i]

                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                #query = {"numero": 44920}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)

                try:
                    # Limpeza para pegar o id do projeto dentro da prefeitura
                    retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                    #tramitacao_encerrada = re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:(.+?)\t\t\t\t\t\t', retorno).group(1)
                    if(re.search('FIMTRAMITA&CCEDIL;&ATILDE;O:SIM', retorno)):
                        tramitacao_encerrada = 'SIM'
                    else:
                        tramitacao_encerrada = 'NAO'
                except:
                    tramitacao_encerrada = 'Informacao nao encontrada'
                    notfound_count+=1
            except:
                failed = True

            listagem['Tramitacao_Encerrada'][i] = tramitacao_encerrada
            resposta.close()
            time.sleep(5)
            
        if failed:
            failed_req += 1
        else:
            filed_req = 0

                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Tramitacao_Encerrada'][i] == '':
            count +=1

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}')

In [ ]:
# Preenchimento do campo Aprovado
count = 1
failed_req= 0
failed = False
aprovado = ''
max_failed_req = 5
notfound_count = 0

while count > 0:
    for i in listagem.index:
        if listagem['Aprovado'][i] == '':
            aprovado = ''
            if listagem['Tramitacao_Encerrada'][i] == 'NAO': 
                aprovado = 'TRAMITACAO'

            elif listagem['Codigo_Tipo'][i] not in ('REQ', 'MOC', 'REP', 'PEDIF'):
                if listagem['Situacao'][i] != 'Arquivada' or 'Vetado' not in listagem['Situacao'][i]: # era 'Vetado Totalmente'
                    aprovado = 'SIM'
                else:
                    aprovado = 'NAO'                

            elif listagem['Codigo_Tipo'][i] in ('REQ', 'MOC', 'REP', 'PEDIF'): 
                failed = False

                try:
                    # Variaveis de requisicao
                    id_proj = listagem['Id_Projeto'][i]

                    # Requisicao para o portal de propostas da Camara
                    query = {"num": id_proj}
                    #query = {"numero": 44920}
                    resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)

                    try:
                        # Limpeza para pegar o id do projeto dentro da prefeitura
                        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                        if(re.search('APROVAD', retorno)):
                            aprovado = 'SIM'
                        else:
                            aprovado = 'NAO'
                    except:
                        aprovado = 'Informacao nao encontrada'
                        notfound_count += 1
                except:
                    failed = True

                resposta.close()
                time.sleep(7)
                
                if failed:
                    failed_req += 1
                else:
                    filed_req = 0

            listagem['Aprovado'][i] = aprovado
            
                
#       Se houve 5 falhas consecutivas
        if failed_req > max_failed_req:
            break
            
    if failed_req > max_failed_req:
        break               
            
    count = 0          

    for i in listagem.index:
        if listagem['Aprovado'][i] == '':
            count +=1

if failed_req > max_failed_req:
    print(f'O endpoint nao esta respondendo, verifique a conexao. Fim do processo. Nao encontrados = {notfound_count}')
else:
    print(f'Fim do processo. Nao encontrados = {notfound_count}')